In [1]:
import requests
import csv
import os
from collections import defaultdict
from dotenv import load_dotenv
from time import sleep
import json

load_dotenv()
API_KEY = os.getenv("API_KEY")

sido_codes = {
    "11": "서울특별시",
    "26": "부산광역시",
    "27": "대구광역시",
    "28": "인천광역시",
    "29": "광주광역시",
    "30": "대전광역시",
    "31": "울산광역시",
    "36": "세종특별자치시",
    "41": "경기도",
    "51": "강원특별자치도",
    "43": "충청북도",
    "44": "충청남도",
    "52": "전라북도",
    "46": "전라남도",
    "47": "경상북도",
    "48": "경상남도",
    "50": "제주특별자치도"
}

years = range(2012, 2025)
months = range(1, 13)

# 용도 목록
usage_types = ["가로등", "교육용", "농사용", "산업용", "일반용", "주택용", "심야"]

# key: (year, metro) → value: dict of usage types + total
aggregated = defaultdict(lambda: defaultdict(int))

for year in years:
    for metroCd, metro_name in sido_codes.items():
        for month in months:
            print(f"📡 Fetching {year}-{month:02d} | {metro_name} ...")
            url = "https://bigdata.kepco.co.kr/openapi/v1/powerUsage/contractType.do"
            params = {
                "year": str(year),
                "month": f"{month:02d}",
                "metroCd": metroCd,
                "apiKey": API_KEY,
                "returnType": "json"
            }

            try:
                response = requests.get(url, params=params)
                raw_text = response.text

                json_start_index = raw_text.find("{")
                if json_start_index != -1:
                    raw_text = raw_text[json_start_index:]

                result = json.loads(raw_text)
                if "data" not in result:
                    print(f"data 필드 없음 at {year}-{month:02d} {metro_name}")
                    continue

                for item in result["data"]:
                    if item["metro"] != metro_name:
                        continue

                    cntr = item["cntr"]
                    usage = int(item["powerUsage"])
                    key = (year, metro_name)

                    if cntr in usage_types:
                        aggregated[key][cntr] += usage
                    else:
                        aggregated[key]["기타"] += usage

                    aggregated[key]["총합"] += usage

            except Exception as e:
                print(f"요청 오류 at {year}-{month:02d} {metro_name}: {e}")

            sleep(1)

# CSV 저장
# CSV 저장
with open("연도별 지역별 전력 사용량.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    headers = ["year", "metro", "metroCd"] + usage_types + ["기타", "총합"]
    writer.writerow(headers)

    for (year, metro), usage_data in sorted(aggregated.items()):
        # metroCd를 sido_codes 딕셔너리에서 역으로 찾기
        metroCd = next((code for code, name in sido_codes.items() if name == metro), "NA")
        row = [year, metro, metroCd]
        for usage_type in usage_types:
            row.append(usage_data.get(usage_type, 0))
        row.append(usage_data.get("기타", 0))
        row.append(usage_data.get("총합", 0))
        writer.writerow(row)


print("연도별 지역별 전력 사용량.csv 저장됨")


📡 Fetching 2012-01 | 서울특별시 ...
📡 Fetching 2012-02 | 서울특별시 ...
📡 Fetching 2012-03 | 서울특별시 ...
📡 Fetching 2012-04 | 서울특별시 ...
📡 Fetching 2012-05 | 서울특별시 ...
📡 Fetching 2012-06 | 서울특별시 ...
📡 Fetching 2012-07 | 서울특별시 ...
📡 Fetching 2012-08 | 서울특별시 ...
📡 Fetching 2012-09 | 서울특별시 ...
📡 Fetching 2012-10 | 서울특별시 ...
📡 Fetching 2012-11 | 서울특별시 ...
📡 Fetching 2012-12 | 서울특별시 ...
📡 Fetching 2012-01 | 부산광역시 ...
📡 Fetching 2012-02 | 부산광역시 ...
📡 Fetching 2012-03 | 부산광역시 ...
📡 Fetching 2012-04 | 부산광역시 ...
📡 Fetching 2012-05 | 부산광역시 ...
📡 Fetching 2012-06 | 부산광역시 ...
📡 Fetching 2012-07 | 부산광역시 ...
📡 Fetching 2012-08 | 부산광역시 ...
📡 Fetching 2012-09 | 부산광역시 ...
📡 Fetching 2012-10 | 부산광역시 ...
📡 Fetching 2012-11 | 부산광역시 ...
📡 Fetching 2012-12 | 부산광역시 ...
📡 Fetching 2012-01 | 대구광역시 ...
📡 Fetching 2012-02 | 대구광역시 ...
📡 Fetching 2012-03 | 대구광역시 ...
📡 Fetching 2012-04 | 대구광역시 ...
📡 Fetching 2012-05 | 대구광역시 ...
📡 Fetching 2012-06 | 대구광역시 ...
📡 Fetching 2012-07 | 대구광역시 ...
📡 Fetching 2012-08 | 대구광역시 ...
📡 Fetchi

In [3]:
import pandas as pd
#지역별 발전 설비
df = pd.read_csv('연도별 지역별 전력 사용량.csv')

print(df)

     year    metro  metroCd        가로등         교육용         농사용          산업용  \
0    2012  강원특별자치도       51  172196667   293051224   441005802   7061175741   
1    2012      경기도       41  738842083  1514010541  1997824531  52601767462   
2    2012     경상남도       48  243075999   416176248  1621659109  19564199999   
3    2012     경상북도       47  242890462   559306829  1405384675  32773042358   
4    2012    광주광역시       29   71806470   275833084    71706553   3027236444   
..    ...      ...      ...        ...         ...         ...          ...   
216  2024     전라남도       46  235218578   349036417  4197272144  21067940683   
217  2024     전라북도       52   17774791    41969373   179436752    999133046   
218  2024  제주특별자치도       50   47379720   152831516  1482183904    685461556   
219  2024     충청남도       44  178329896   506401417  2697906257  36587401447   
220  2024     충청북도       43  182397517   346871629  1042254843  18213696354   

             일반용          주택용  심야          기타      